# FINMA Python Lab 07 - Solutions

This notebook contains complete solutions for all exercises in Lab 7.

---

## Exercise 1: Count Words in Market Report

In [ ]:
def count_words_in_report(filename):
    """
    Count total words and specific keywords in market report.
    """
    keywords = ['market', 'sector', 'trading']
    keyword_counts = {kw: 0 for kw in keywords}
    total_words = 0
    
    with open(filename, 'r') as file:
        for line in file:
            # Split line into words and convert to lowercase
            words = line.lower().split()
            total_words += len(words)
            
            # Count keywords
            for word in words:
                # Remove punctuation
                word = word.strip('.,!?:;')
                if word in keywords:
                    keyword_counts[word] += 1
    
    # Print results
    print(f"Total words: {total_words}")
    print("\nKeyword counts:")
    for keyword, count in keyword_counts.items():
        print(f"  {keyword}: {count}")
    
    return total_words, keyword_counts

# Test
count_words_in_report('sample_data/market_report.txt')

---

## Exercise 2: Extract Holdings from Markdown

In [ ]:
def extract_holdings(filename):
    """
    Extract stock symbols from markdown file.
    Symbols are marked with ** bold markers.
    """
    symbols = []
    
    with open(filename, 'r') as file:
        for line in file:
            # Look for patterns like **AAPL**
            if '**' in line:
                # Split by ** and get every other element
                parts = line.split('**')
                for i in range(1, len(parts), 2):
                    symbol = parts[i].strip()
                    # Check if it's a valid stock symbol (all caps, 1-5 letters)
                    if symbol.isupper() and 1 <= len(symbol) <= 5:
                        symbols.append(symbol)
    
    print(f"Holdings found: {symbols}")
    return symbols

# Test
holdings = extract_holdings('sample_data/portfolio_report.md')

---

## Exercise 3: Calculate Sector Totals

In [ ]:
import csv

def calculate_sector_totals(filename):
    """
    Calculate total value for each sector from portfolio.
    """
    sector_totals = {}
    
    with open(filename, 'r') as file:
        csv_reader = csv.DictReader(file)
        
        for row in csv_reader:
            sector = row['sector']
            value = float(row['value'])
            
            if sector not in sector_totals:
                sector_totals[sector] = 0
            sector_totals[sector] += value
    
    # Sort by value (descending)
    sorted_sectors = sorted(sector_totals.items(), key=lambda x: x[1], reverse=True)
    
    # Print results
    print("Sector Totals:")
    print("=" * 50)
    for sector, total in sorted_sectors:
        print(f"{sector:30} ${total:12,.2f}")
    
    return sector_totals

# Test
sector_totals = calculate_sector_totals('sample_data/portfolio.csv')

---

## Exercise 4: Price Change Analysis

In [ ]:
import csv

def analyze_price_changes(input_file, output_file):
    """
    Calculate price changes from first to last day for each stock.
    """
    # Track first and last prices for each symbol
    price_data = {}
    
    with open(input_file, 'r') as file:
        csv_reader = csv.DictReader(file)
        
        for row in csv_reader:
            symbol = row['symbol']
            close = float(row['close'])
            
            if symbol not in price_data:
                price_data[symbol] = {'first': close, 'last': close}
            else:
                price_data[symbol]['last'] = close
    
    # Calculate changes and write to file
    results = []
    for symbol, prices in price_data.items():
        start = prices['first']
        end = prices['last']
        change = end - start
        change_pct = (change / start) * 100
        
        results.append({
            'symbol': symbol,
            'start_price': start,
            'end_price': end,
            'change': change,
            'change_pct': change_pct
        })
    
    # Write to CSV
    with open(output_file, 'w', newline='') as file:
        fieldnames = ['symbol', 'start_price', 'end_price', 'change', 'change_pct']
        csv_writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        csv_writer.writeheader()
        csv_writer.writerows(results)
    
    print(f"Price change analysis written to {output_file}")
    
    # Print summary
    print("\nPrice Changes:")
    for result in results:
        print(f"{result['symbol']}: ${result['change']:.2f} ({result['change_pct']:+.2f}%)")
    
    return results

# Test
analyze_price_changes('sample_data/stock_prices.csv', 'output/price_changes.csv')

---

## Exercise 5: Portfolio Performance Report

In [ ]:
import csv
from datetime import datetime

def generate_performance_report(input_file, output_file):
    """
    Generate comprehensive portfolio performance report.
    """
    # Read portfolio
    holdings = []
    with open(input_file, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            row['value'] = float(row['value'])
            holdings.append(row)
    
    # Calculate statistics
    total_value = sum(h['value'] for h in holdings)
    num_positions = len(holdings)
    avg_position = total_value / num_positions
    
    # Find extremes
    largest = max(holdings, key=lambda x: x['value'])
    smallest = min(holdings, key=lambda x: x['value'])
    
    # Write report
    with open(output_file, 'w') as file:
        file.write("PORTFOLIO PERFORMANCE REPORT\n")
        file.write("="*50 + "\n")
        file.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        file.write("Summary Statistics:\n")
        file.write(f"  Total Value: ${total_value:,.2f}\n")
        file.write(f"  Number of Positions: {num_positions}\n")
        file.write(f"  Average Position: ${avg_position:,.2f}\n\n")
        
        file.write("Extremes:\n")
        file.write(f"  Largest: {largest['symbol']} (${largest['value']:,.2f})\n")
        file.write(f"  Smallest: {smallest['symbol']} (${smallest['value']:,.2f})\n")
    
    print(f"Report written to {output_file}")
    
    # Display the report
    with open(output_file, 'r') as file:
        print("\n" + file.read())

# Test
generate_performance_report('sample_data/portfolio.csv', 'output/performance_report.txt')

---

## Exercise 6: Filter and Export

In [ ]:
import csv

def filter_portfolio(input_file, output_file, sector=None, min_value=0):
    """
    Filter portfolio based on sector and minimum value.
    """
    filtered_holdings = []
    
    # Read and filter
    with open(input_file, 'r') as file:
        csv_reader = csv.DictReader(file)
        
        for row in csv_reader:
            value = float(row['value'])
            
            # Apply filters
            if sector and row['sector'] != sector:
                continue
            if value < min_value:
                continue
            
            filtered_holdings.append(row)
    
    # Write filtered results
    if filtered_holdings:
        with open(output_file, 'w', newline='') as file:
            fieldnames = filtered_holdings[0].keys()
            csv_writer = csv.DictWriter(file, fieldnames=fieldnames)
            
            csv_writer.writeheader()
            csv_writer.writerows(filtered_holdings)
    
    print(f"Filtered {len(filtered_holdings)} holdings to {output_file}")
    print("\nFiltered Holdings:")
    for holding in filtered_holdings:
        print(f"{holding['symbol']:6} | {holding['company']:25} | ${float(holding['value']):,.2f}")
    
    return len(filtered_holdings)

# Test
count = filter_portfolio(
    'sample_data/portfolio.csv',
    'output/tech_holdings.csv',
    sector='Technology',
    min_value=10000
)
print(f"\nTotal filtered: {count}")

---

## Exercise 7: Combine Multiple Files

In [ ]:
import csv

def combine_portfolio_with_prices(portfolio_file, prices_file, output_file):
    """
    Combine portfolio with latest prices to show updated values.
    """
    # Get latest prices for each symbol
    latest_prices = {}
    with open(prices_file, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            symbol = row['symbol']
            latest_prices[symbol] = float(row['close'])
    
    # Read portfolio and calculate updated values
    results = []
    with open(portfolio_file, 'r') as file:
        csv_reader = csv.DictReader(file)
        
        for row in csv_reader:
            symbol = row['symbol']
            shares = int(row['shares'])
            original_price = float(row['price'])
            original_value = float(row['value'])
            
            # Get current price if available
            current_price = latest_prices.get(symbol, original_price)
            current_value = shares * current_price
            change = current_value - original_value
            
            results.append({
                'symbol': symbol,
                'shares': shares,
                'original_price': original_price,
                'current_price': current_price,
                'original_value': original_value,
                'current_value': current_value,
                'change': change
            })
    
    # Write to CSV
    with open(output_file, 'w', newline='') as file:
        fieldnames = ['symbol', 'shares', 'original_price', 'current_price',
                     'original_value', 'current_value', 'change']
        csv_writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        csv_writer.writeheader()
        csv_writer.writerows(results)
    
    print(f"Combined data written to {output_file}")
    print("\nUpdated Portfolio:")
    print("=" * 80)
    
    total_change = 0
    for result in results:
        print(f"{result['symbol']:6} | "
              f"${result['original_value']:10,.2f} -> "
              f"${result['current_value']:10,.2f} | "
              f"Change: ${result['change']:+,.2f}")
        total_change += result['change']
    
    print("=" * 80)
    print(f"Total Change: ${total_change:+,.2f}")
    
    return results

# Test
combine_portfolio_with_prices(
    'sample_data/portfolio.csv',
    'sample_data/stock_prices.csv',
    'output/updated_portfolio.csv'
)

---

## Exercise 8: Trading Log System

In [ ]:
import csv
from datetime import datetime
import os

def add_trade(filename, date, symbol, action, shares, price):
    """
    Add a trade to the trading log.
    """
    # Check if file exists to determine if we need to write header
    file_exists = os.path.exists(filename)
    
    with open(filename, 'a', newline='') as file:
        csv_writer = csv.writer(file)
        
        # Write header if new file
        if not file_exists:
            csv_writer.writerow(['date', 'symbol', 'action', 'shares', 'price'])
        
        # Write trade
        csv_writer.writerow([date, symbol, action, shares, price])
    
    print(f"Trade added: {action} {shares} {symbol} @ ${price}")

def read_trades(filename):
    """
    Read all trades from file.
    """
    trades = []
    
    if not os.path.exists(filename):
        return trades
    
    with open(filename, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            row['shares'] = int(row['shares'])
            row['price'] = float(row['price'])
            trades.append(row)
    
    return trades

def calculate_pl(trades):
    """
    Calculate profit/loss for each symbol.
    """
    positions = {}
    
    for trade in trades:
        symbol = trade['symbol']
        shares = trade['shares']
        price = trade['price']
        action = trade['action']
        
        if symbol not in positions:
            positions[symbol] = {'shares': 0, 'cost': 0, 'proceeds': 0}
        
        if action == 'BUY':
            positions[symbol]['shares'] += shares
            positions[symbol]['cost'] += shares * price
        elif action == 'SELL':
            positions[symbol]['shares'] -= shares
            positions[symbol]['proceeds'] += shares * price
    
    # Calculate P&L
    pl_by_symbol = {}
    for symbol, pos in positions.items():
        realized_pl = pos['proceeds'] - (pos['cost'] * pos['proceeds'] / (pos['cost'] + pos['proceeds']) if pos['cost'] + pos['proceeds'] > 0 else 0)
        pl_by_symbol[symbol] = realized_pl
    
    return pl_by_symbol

def generate_summary(trades, output_file):
    """
    Generate trading summary report.
    """
    if not trades:
        print("No trades to summarize")
        return
    
    # Calculate statistics
    total_trades = len(trades)
    buy_volume = sum(t['shares'] * t['price'] for t in trades if t['action'] == 'BUY')
    sell_volume = sum(t['shares'] * t['price'] for t in trades if t['action'] == 'SELL')
    pl_by_symbol = calculate_pl(trades)
    total_pl = sum(pl_by_symbol.values())
    
    # Write summary
    with open(output_file, 'w') as file:
        file.write("TRADING SUMMARY REPORT\n")
        file.write("=" * 50 + "\n")
        file.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        file.write("Overall Statistics:\n")
        file.write(f"  Total Trades: {total_trades}\n")
        file.write(f"  Total Buy Volume: ${buy_volume:,.2f}\n")
        file.write(f"  Total Sell Volume: ${sell_volume:,.2f}\n\n")
        
        file.write("P&L by Symbol:\n")
        file.write("-" * 50 + "\n")
        for symbol, pl in sorted(pl_by_symbol.items()):
            file.write(f"  {symbol:6}: ${pl:+,.2f}\n")
        
        file.write("-" * 50 + "\n")
        file.write(f"  Total P&L: ${total_pl:+,.2f}\n")
    
    print(f"Summary written to {output_file}")
    
    # Display summary
    with open(output_file, 'r') as file:
        print("\n" + file.read())

# Test the system
trade_file = 'output/trades_log.csv'

# Add some trades
add_trade(trade_file, '2026-01-02', 'AAPL', 'BUY', 100, 150.00)
add_trade(trade_file, '2026-01-03', 'AAPL', 'SELL', 50, 152.00)
add_trade(trade_file, '2026-01-02', 'GOOGL', 'BUY', 50, 138.00)
add_trade(trade_file, '2026-01-04', 'GOOGL', 'SELL', 25, 140.00)

# Read and generate summary
trades = read_trades(trade_file)
generate_summary(trades, 'output/trading_summary.txt')